In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_line, get_post_df
from codebase.ibis import post_process_sign
import altair as alt
from codebase.classes_data import Data
from run_mcmc import run_mcmc
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir1 = 'log/20210202_192758_mcmc/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [3]:
log_dir2 = 'log/20210202_195936_hmc/'
exp_data2 = load_obj( 'complete_data', log_dir2)
ps_hmc = load_obj('hmc_post_samples', log_dir2)


In [4]:
ps_hmc['beta'].shape

(1000, 6, 1)

##

## Or load existing directory

In [5]:
# log_dir = '/log/20210122_173348_mcmc_ibis_adapt9/'
# ps = load_obj( 'mcmc_post_samples', log_dir)
# ps['beta'] = ps['beta'][num_warmup:].copy()

## Post process loadings for sign flips

In [6]:
ps_hmc = post_process_sign(ps_hmc)
ps_mcmc = post_process_sign(ps_mcmc)

## Plot MCMC samples

In [9]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'
plot_line(df, height=100)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [10]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'

df2 = get_post_df(ps_hmc[param]) 
df2['source'] = 'hmc'

plot_density(pd.concat([df,df2]), height=100)

alt.Chart(...)

In [11]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'

df2 = get_post_df(ps_hmc[param]) 
df2['source'] = 'hmc'

plot_density(pd.concat([df,df2]), height=100)

alt.Chart(...)

## Plot quantiles

In [17]:
param = 'beta'
df = get_post_df(ps_mcmc[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'mcmc'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [18]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'mcmc'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [19]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)